# CSCI 3352 Project

**Names:** Tom Stoughton, Dylan Smith

Summary: ...

In [12]:
# network/comp. bio. packages
import networkx as nx
import cobra
from cobra.io import read_sbml_model
import netwulf
from netwulf import visualize

# helper packages
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

The following function is used to generate a metabolic interaction network from the SBML model of different genomes in the Mycoplasma genus.

In [13]:
# set up networks here

# iterate over model.reactions and model.metabolites to construct bipartite directed network
def makeNetworkFromSBML(model):
    G = nx.DiGraph()
    nodes_m = {}
    nodes_r = {}

    for metabolite in model.metabolites:
        nodes_m[metabolite.id] = metabolite.formula
        
    for reaction in model.reactions:
        nodes_r[reaction.id] = reaction.name
    
    # add nodes with metabolite formula/enzyme name as attribute for id
    G.add_nodes_from([(m_id, {'formula': formula}) for (m_id, formula) in nodes_m.items()], bipartite=0)
    G.add_nodes_from([(r_id, {'name': name}) for (r_id, name) in nodes_r.items()], bipartite=1)
    
    edges = []
    for reaction in model.reactions:
        # get products and reactants
        products  = reaction.products
        reactants = reaction.reactants
        enzyme = reaction.name
        rid = reaction.id
        
        for p in products:
            edges.append((rid, p.id))
            if reaction.reversibility:
                edges.append((p.id, rid))
                
        for r in reactants:
            edges.append((r.id, rid))
            if reaction.reversibility:
                edges.append((rid, r.id))
            
        
    G.add_edges_from(edges)
    return G


In [28]:
# create list of metabolic networks, one for each genome
networks = {}
sbml_dir = 'sbml_files'
for sbml_file in os.listdir(sbml_dir):
    # print(sbml_file)
    model = read_sbml_model(os.path.join(sbml_dir, sbml_file))
    networks[sbml_file] = makeNetworkFromSBML(model)

In [29]:
for f, n in networks.items():
    print(f,':', n)

tullyi_gcf.sbml : DiGraph with 608 nodes and 2286 edges
JCVI.sbml : DiGraph with 609 nodes and 2302 edges
procyoni_gcf.sbml : DiGraph with 608 nodes and 2281 edges
buteonis_gcf.sbml : DiGraph with 608 nodes and 2294 edges
marinum_gcf.sbml : DiGraph with 608 nodes and 2291 edges
enhydrae_gcf.sbml : DiGraph with 609 nodes and 2313 edges
haemobos.sbml : DiGraph with 611 nodes and 2312 edges
struthionis_gcf.sbml : DiGraph with 609 nodes and 2304 edges
zalophi_gcf.sbml : DiGraph with 611 nodes and 2306 edges
cottewii_gcf.sbml : DiGraph with 611 nodes and 2308 edges
feriruminatoris_gcf.sbml : DiGraph with 608 nodes and 2309 edges


In [ ]:
# clustering analysis